Install necessary libraries

In [3]:
!pip -q install langchain openai tiktoken pypdf faiss-cpu langchainhub
!pip install -U langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.5/238.5 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


Chain with chat history

ASTRA DB as database
One can choose any based on the requirement

Available options:
"AstraDBChatMessageHistory",
    "ChatMessageHistory",
    "CassandraChatMessageHistory",
    "CosmosDBChatMessageHistory",
    "DynamoDBChatMessageHistory",
    "ElasticsearchChatMessageHistory",
    "FileChatMessageHistory",
    "FirestoreChatMessageHistory",
    "MomentoChatMessageHistory",
    "MongoDBChatMessageHistory",
    "PostgresChatMessageHistory",
    "RedisChatMessageHistory",
    "RocksetChatMessageHistory",
    "SQLChatMessageHistory",
    "StreamlitChatMessageHistory",
    "SingleStoreDBChatMessageHistory",
    "XataChatMessageHistory",
    "ZepChatMessageHistory",
    "UpstashRedisChatMessageHistory",
    "Neo4jChatMessageHistory",
    "TiDBChatMessageHistory",

In [1]:
%pip install --upgrade --quiet  "astrapy>=0.6.2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00


In [ ]:
import getpass

ASTRA_DB_API_ENDPOINT = input("ASTRA_DB_API_ENDPOINT = ")
ASTRA_DB_APPLICATION_TOKEN = getpass.getpass("ASTRA_DB_APPLICATION_TOKEN = ")

Declare OPENAI API KEY

In [22]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [14]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnableLambda
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

from langchain.schema import format_document
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from operator import itemgetter
from langchain.memory import ConversationBufferMemory

In [15]:
PATH = "/content/1.pdf" #your PDF path

loader = PyPDFLoader(PATH)
pages = loader.load_and_split()

#pages will contain metadata like page and source pdf as well

Now split the full text into chunks

In [16]:
# Splitting up the text into smaller chunks for indexing
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200, #striding over the text
    length_function = len,
)


In [17]:
docs_new = text_splitter.split_documents(pages)


In [18]:
len(docs_new)

49

In [19]:
docs_new[17].page_content

'ChengduEbyteElectronicTechnologyCo,;Ltd E32-433T30Dusermanual\nCopyright©2012–2019，ChengduEbyteElectronicTechnologyCo.,Ltd. 8\n5Functiondescription\n5.1Fixedtransmission\n5.2Broadcastingtransmission\n5.3Broadcastingaddress\n\uf06cForexample:SettheaddressofmoduleAas0xFFFF,andthechannelas0x04;\n\uf06cWhenmoduleisthetransmitter(transparenttransmission),allmodulesunderchannel0x04willreceivethedata,the'

In [20]:
docs_new[17].metadata

{'source': '/content/1.pdf', 'page': 8}

Now Embeddings of the data

In [23]:
embeddings = OpenAIEmbeddings()

In [24]:
db = FAISS.from_documents(docs_new, embeddings)

Retriever

In [25]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [ ]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7bdb35607e20>, search_kwargs={'k': 6})

In [9]:
from langchain.prompts.prompt import PromptTemplate

_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.
If chat history is empty please don't change the question

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

condense_question_prompt = PromptTemplate.from_template(_template)

In [10]:
template =  """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise. context:
{context}

Question: {question}
"""

answer_prompt = ChatPromptTemplate.from_template(template)

default_doc_template =PromptTemplate.from_template(template = '{page_content}')

In [12]:
def _combine_documents(
    docs, document_prompt=default_doc_template, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [28]:
#if want to use local ConversationBufferMemory buffer:
memory = ConversationBufferMemory(
    return_messages = True, output_key = "answer", input_key = "question",
)

In [31]:
#or if you want to save it on cloud :
from langchain.memory import AstraDBChatMessageHistory

message_history = AstraDBChatMessageHistory(
    session_id="test-session",
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
)

memory = ConversationBufferMemory(
        memory_key="history",
        chat_memory=message_history,
        return_messages=True,
        output_key="answer",
        input_key="question"
    )




In [44]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi!'), AIMessage(content='whats up?')]}

In [32]:


loaded_memory = RunnablePassthrough.assign(
    chat_history = RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)

In [54]:
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | condense_question_prompt
    | ChatOpenAI(temperature=0)
    | StrOutputParser(),

}

retrieved_docs = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}

final_inputs = {
    "context":lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | answer_prompt | ChatOpenAI(temperature=0),
    "docs": itemgetter("docs"),
}


final_chain = loaded_memory | standalone_question | retrieved_docs | answer

Invoke chain

In [56]:
inputs = {"question": "Who manufature E32?"}
result = final_chain.invoke(inputs)
print(result)

{'answer': AIMessage(content='Chengdu Ebyte Electronic Technology Co., Ltd manufactures E32.'), 'docs': [Document(page_content='E32-915T30DSX1276915M 30 80.3~19.2kDIP24*43SMA-K\nE32-170T30DSX1278170M 30 80.3k~9.6kDIP24*43SMA-K\nE32-868T20DSX1276868M 20 30.3~19.2kDIP21*36SMA-K\nE32-915T20DSX1276915M 20 30.3~19.2kDIP21*36SMA-K\nE32-433T20DCSX1278433M 20 30.3k~19.2kDIP21*36SMA-K\nE32-433T30DSX1278433M 30 80.3k~19.2kDIP24*43SMA-K', metadata={'source': '/content/1.pdf', 'page': 18}), Document(page_content='ChengduEbyteElectronicTechnologyCo,;Ltd E32-433T30Dusermanual\nCopyright©2012–2019，ChengduEbyteElectronicTechnologyCo.,Ltd. 3\nDisclaimer\nEBYTEreservesallrightstothisdocumentandtheinformationcontainedherein.\nProducts,names,logosanddesignsdescribedhereinmayinwholeorinpartbe\nsubjecttointellectualpropertyrights.Reproduction,use,modificationordisclosureto\nthirdpartiesofthisdocumentoranypartthereofwithouttheexpresspermissionof\nEBYTEisstrictlyprohibited.\nTheinformationcontainedhereinispro

In [57]:
memory.save_context(inputs, {"answer": result["answer"].content})

In [58]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi!'),
  AIMessage(content='whats up?'),
  HumanMessage(content='What is E32 ?'),
  AIMessage(content='The given context does not provide information about what "E32" means.'),
  HumanMessage(content='Who manufature E32?'),
  AIMessage(content='Chengdu Ebyte Electronic Technology Co., Ltd manufactures E32.')]}